In [ ]:
!pip install boto3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 880.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.6 MB/s eta 0:00:00


In [ ]:
!pip install awscli


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.2/548.2 kB 15.7 MB/s eta 0:00:00
  Attempting uninstall: rsa
    Found existing installation: rsa 4.9
    Uninstalling rsa-4.9:
      Successfully uninstalled rsa-4.9
  Attempting uninstall: docutils
    Found existing installation: docutils 0.21.2
    Uninstalling docutils-0.21.2:
      Successfully uninstalled docutils-0.21.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [ ]:
import os
import boto3
import pandas as pd

In [ ]:
import time

In [ ]:


# Set environment variables
os.environ['AWS_ACCESS_KEY_ID'] = ' ' #paste ID
os.environ['AWS_SECRET_ACCESS_KEY'] = ' ' # paste key


# Initialize the Boto3 client using the environment variables
client = boto3.client(service_name='comprehendmedical', region_name='us-east-1')


In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataframe = pd.read_csv('drive/My Drive/Projects/LSE-Symptomatology/Dataset/Sampled_Posts_1000.csv')

In [ ]:
dataframe=dataframe.astype(str)
dataframe = dataframe.replace('\n','', regex=True)

In [ ]:
dataframe

,postID,post,word_count,category
0,xapb9j,I forgot to take my pill yesterday. Is it okay...,51,Group 1
1,va1s9b,how do i lose weight?. i’ve gained a lot of we...,84,Group 1
2,vyv2ut,Lab results. I just got my labs back and I’d l...,45,Group 1
3,wy5f9k,Can you get rid of the follicles/cysts?. No ot...,61,Group 1
4,yvjy63,hair loss. i’m slowly balding from my scalp. i...,43,Group 1
...,...,...,...,...
995,y87em5,PCOS Beginner Questions. Hi all! I don’t have ...,344,Group 4
996,v96ws6,Insulin Resistance/Sugar Readings. Hello! I wa...,353,Group 4
997,yj6zyi,Weird coincidence??. So I have no idea what’s ...,301,Group 4
998,y6k0n6,Hormone irregularity. I’ll try not to make thi...,346,Group 4


In [ ]:
df_corpus = pd.DataFrame()
df_corpus['sample_text'] = dataframe['post']

In [ ]:
df_corpus

,sample_text
0,I forgot to take my pill yesterday. Is it okay...
1,how do i lose weight?. i’ve gained a lot of we...
2,Lab results. I just got my labs back and I’d l...
3,Can you get rid of the follicles/cysts?. No ot...
4,hair loss. i’m slowly balding from my scalp. i...
...,...
995,PCOS Beginner Questions. Hi all! I don’t have ...
996,Insulin Resistance/Sugar Readings. Hello! I wa...
997,Weird coincidence??. So I have no idea what’s ...
998,Hormone irregularity. I’ll try not to make thi...


In [ ]:
def extract_medical_conditions(text):
    try:
        result = client.detect_entities_v2(Text=text)
        entities = result['Entities']
        medical_conditions = [entity['Text'] for entity in entities if entity['Category'] == 'MEDICAL_CONDITION']
        return ', '.join(medical_conditions)
    except Exception as e:
        print(f"Error processing text: {text}\nError: {str(e)}")
        return ''


In [ ]:
def process_text_in_batches(df, batch_size=20, delay=30):

    df = df.copy()
    results = []
    for i in range(0, len(df), batch_size):
        # Extract the current batch
        batch = df.iloc[i:i + batch_size]

        batch_results = batch['sample_text'].apply(extract_medical_conditions)
        results.extend(batch_results)

        # Add a delay
        if i + batch_size < len(df):
            print(f"Processed rows {i} to {i + batch_size}. Waiting for {delay} seconds...")
            time.sleep(delay)


    df['medical_condition'] = results
    return df

In [ ]:
# Parameters
chunk_size = 100  # Number of rows per chunk
batch_size = 20   # Number of rows to process per batch
delay = 30        # Delay
total_rows = len(df_corpus)

In [ ]:
all_results = []

In [ ]:
# Process 1000 rows in chunks of 100, with 20 rows per batch within each chunk
for start_row in range(0, total_rows, chunk_size):
    end_row = min(start_row + chunk_size, total_rows)
    print(f"Processing rows {start_row} to {end_row}...")

    # Get the current chunk and process in batches
    chunk = df_corpus.iloc[start_row:end_row].copy()
    processed_chunk = process_text_in_batches(chunk, batch_size=batch_size, delay=delay)

    # Append processed chunk to results
    all_results.append(processed_chunk)

Processing rows 0 to 100...
Processed rows 0 to 20. Waiting for 30 seconds...
Processed rows 20 to 40. Waiting for 30 seconds...
Processed rows 40 to 60. Waiting for 30 seconds...
Processed rows 60 to 80. Waiting for 30 seconds...
Processing rows 100 to 200...
Processed rows 0 to 20. Waiting for 30 seconds...
Processed rows 20 to 40. Waiting for 30 seconds...
Processed rows 40 to 60. Waiting for 30 seconds...
Processed rows 60 to 80. Waiting for 30 seconds...
Processing rows 200 to 300...
Processed rows 0 to 20. Waiting for 30 seconds...
Processed rows 20 to 40. Waiting for 30 seconds...
Processed rows 40 to 60. Waiting for 30 seconds...
Processed rows 60 to 80. Waiting for 30 seconds...
Processing rows 300 to 400...
Processed rows 0 to 20. Waiting for 30 seconds...
Processed rows 20 to 40. Waiting for 30 seconds...
Processed rows 40 to 60. Waiting for 30 seconds...
Processed rows 60 to 80. Waiting for 30 seconds...
Processing rows 400 to 500...
Processed rows 0 to 20. Waiting for 30 s

In [ ]:
# Combine all chunks into a DataFrame
final_df = pd.concat(all_results, ignore_index=True)

In [ ]:
final_df

,sample_text,medical_condition
0,I forgot to take my pill yesterday. Is it okay...,side effects
1,how do i lose weight?. i’ve gained a lot of we...,"gained a lot of weight, stabilized, very unhea..."
2,Lab results. I just got my labs back and I’d l...,
3,Can you get rid of the follicles/cysts?. No ot...,"follicles, cysts, symptoms, follicles, cysts, ..."
4,hair loss. i’m slowly balding from my scalp. i...,"hair loss, balding, affecting my mental health..."
...,...,...
995,PCOS Beginner Questions. Hi all! I don’t have ...,"PCOS, hair loss on scalp, weight gain, acne, m..."
996,Insulin Resistance/Sugar Readings. Hello! I wa...,"PCOS, irregular periods, nervous, anxious, aff..."
997,Weird coincidence??. So I have no idea what’s ...,"PCOS, excess hair growth, acne, irregular peri..."
998,Hormone irregularity. I’ll try not to make thi...,"Hormone irregularity, gaining, gaining over 10..."


In [ ]:
final_df['unique_medical_condition'] = final_df['medical_condition'].apply(
    lambda x: "none" if not x or x.strip() == "" else ", ".join(sorted(set(x.split(", "))))
)

In [ ]:
final_df

,sample_text,medical_condition,unique_medical_condition
0,I forgot to take my pill yesterday. Is it okay...,side effects,side effects
1,how do i lose weight?. i’ve gained a lot of we...,"gained a lot of weight, stabilized, very unhea...","gained a lot of weight, gaining weight, losing..."
2,Lab results. I just got my labs back and I’d l...,,none
3,Can you get rid of the follicles/cysts?. No ot...,"follicles, cysts, symptoms, follicles, cysts, ...","cysts, follicles, hormones fine, regular perio..."
4,hair loss. i’m slowly balding from my scalp. i...,"hair loss, balding, affecting my mental health...","affecting my mental health, balding, hair loss"
...,...,...,...
995,PCOS Beginner Questions. Hi all! I don’t have ...,"PCOS, hair loss on scalp, weight gain, acne, m...","PCOS, acne, anxiety, cramps, depression, doubl..."
996,Insulin Resistance/Sugar Readings. Hello! I wa...,"PCOS, irregular periods, nervous, anxious, aff...","PCOS, affect my fertility, anxious, circulatio..."
997,Weird coincidence??. So I have no idea what’s ...,"PCOS, excess hair growth, acne, irregular peri...","IBS, PCOS, acne, bloat, bloated, bowel movemen..."
998,Hormone irregularity. I’ll try not to make thi...,"Hormone irregularity, gaining, gaining over 10...","EXHAUSTED, Hormone irregularity, Hot flashes, ..."


In [ ]:
print(final_df['sample_text'].loc[final_df.index[999]])

Need advice - Low Libido. Hi everyone!I am 22 yrs old and was recently diagnosed with PCOS about 3 month ago. I had very high testosterone from my blood results and I did an ultrasound to confirm that I did have PCOS. I was lurking through this subreddit to see if anyone experienced low libido, I did see a lot of other people experience the same thing, but I wanted to ask some advice, if I should try to do something about it or maybe visit a doctor again to do something about it. I’m beyond frustrated.Some background:I used to have a very high sex drive all throughout my teen years. I noticed a decrease of sex drive exactly two years ago ( ~ Oct 2020) and I did have covid in Sept 2020. I kinda figured it was covid and shrugged it off until I continued to not have a sex drive for months. Any time I’d attempt to have sex I was dry and it would not feel good. I couldn’t go to a doctor at that time because I had no income or insurance.I finally got insurance, I went to the gynecologist, sh

In [ ]:
print(final_df['medical_condition'].loc[final_df.index[250]])

PCOS, PCOS, feel better, periods, normal periods, hair stopped falling out, lose the weight, insulin resistance2


In [ ]:
# Save the sample
final_df.to_excel('drive/My Drive/Projects/LSE-Symptomatology/Dataset/Dataset (Baseline Comparisons)/PCOS Symptoms (Amazon Comprehend Medical).xlsx', index=False)